In [21]:
# =========================
# 1. Imports
# =========================
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [23]:
# =========================
# 2. Charger les données
# =========================
data = pd.read_excel("../hotel_reviews_preprocessed.xlsx")

X = data["review"][:2000]
y = data["Sentiment"].map({"Positive": 1, "Negative": 0})[:2000]

In [24]:
# =========================
# 3. Split Train / Test
# =========================
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [25]:
# =========================
# 4. Modèle CAMeL-BERT
# =========================
MODEL_NAME = "CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2,
    ignore_mismatched_sizes=True
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [26]:
# =========================
# 5. Dataset
# =========================
class ReviewDataset(Dataset):
    def __init__(self, texts, labels):
        self.enc = tokenizer(
            list(texts),
            truncation=True,
            padding=True,
            max_length=64
        )
        self.labels = list(labels)

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.enc.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ReviewDataset(X_train, y_train)
test_dataset  = ReviewDataset(X_test, y_test)

In [27]:
# =========================
# 6. Metrics
# =========================
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="binary"
    )
    acc = accuracy_score(labels, preds)

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [29]:
# =========================
# 7. Entraînement
# =========================
training_args = TrainingArguments(
    output_dir="./results_camelbert",
    num_train_epochs=2,
    per_device_train_batch_size=32,
    learning_rate=2e-5,
    logging_steps=20,
    save_strategy="no",
    eval_strategy="epoch",
    report_to="none",
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()

C:\Users\kader\AppData\Local\Temp\ipykernel_17992\4089507066.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
c:\Users\kader\Desktop\TP_HARD\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.215600,0.182110,0.935000,0.910112,0.941860,0.925714
2,0.115800,0.186962,0.932500,0.905028,0.941860,0.923077


c:\Users\kader\Desktop\TP_HARD\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\kader\Desktop\TP_HARD\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.18696220219135284,
 'eval_accuracy': 0.9325,
 'eval_precision': 0.9050279329608939,
 'eval_recall': 0.9418604651162791,
 'eval_f1': 0.9230769230769231,
 'eval_runtime': 34.6084,
 'eval_samples_per_second': 11.558,
 'eval_steps_per_second': 1.445,
 'epoch': 2.0}

In [30]:
# =========================
# 8. Sauvegarde du modèle
# =========================
SAVE_PATH = "../camelbert_sentiment_model"
trainer.save_model(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)

print("✅ CAMeL-BERT sauvegardé")

✅ CAMeL-BERT sauvegardé


In [31]:
# =========================
# 9. Test rapide
# =========================
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        logits = model(**inputs).logits
    return "Positif" if torch.argmax(logits) == 1 else "Négatif"

print(predict("الفندق ممتاز والخدمة رائعة"))
print(predict("التجربة كانت سيئة جدا"))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Positif
Négatif
